In [38]:
import os
from jsonschema_rs import validator_for
import json

from jsonschema import validate
from pbcore.models.models import PBCore

In [2]:
SCHEMA='schemas/pbcore.schema.json'
DIRECTORY = 'pbcore-json'

with open(SCHEMA, 'r') as schema_file:
    schema = json.load(schema_file)

pb = validator_for(schema)

# with open(os.path.join(DIRECTORY, 'cpb-aacip_15-29p2p5g8.json'), 'r') as f:
with open(os.path.join(DIRECTORY, 'cpb-aacip-15-nc5s756s32.json'), 'r') as f:
    json_data = json.load(f)

# json_data

# validate_json(json_data)
# validate(json_data, schema)
pb.validate(json_data)

In [ ]:
errors = []
success = []
for path, subdirs, files in os.walk(DIRECTORY):
    for name in files:
        if name.startswith('cpb-aacip') and name.endswith('.json'):
            with open(os.path.join(path, name), 'r') as f:
                json_data = json.load(f)
                try:
                    # validate(json_data, schema) # jsonschema
                    # pb.validate(json_data) # jsonschema_rs
                    PBCore.model_validate(json_data) # pydantic
                    # print(f"{name} is valid.")
                    success.append(name)
                except Exception as e:
                    print(f"{name} is invalid: {e.message}")
                    errors.append((name, e.message))


In [10]:
len(success), len(errors)

(4159, 0)

In [34]:
mvp={
        "pbcoreDescriptionDocument": {
            "xsi:schemaLocation": "http://www.example.com/schema",
            "pbcoreIdentifier": [{"text": "123", "source": "example"}],
            "pbcoreTitle": [{"text": "Title"}],
            "pbcoreDescription": [{"text": ""}],
        },
    }
# del mvp['pbcoreDescriptionDocument']['pbcoreIdentifier'][0]['source']

pb.validate(mvp)

In [12]:
def set_nested_path(d, path, value):
    keys = path.split('.')
    for key in keys[:-1]:
        d = d.setdefault(key, {})
    d[keys[-1]] = value
    return d

In [ ]:
from dotmap import DotMap

p = DotMap(mvp)
p.pprint()
pb.validate(p.toDict())


{'pbcoreDescriptionDocumen': {'pbcoreIdentifier': {0: {'text': '123'}}},
 'pbcoreDescriptionDocument': {'pbcoreDescription': [{'text': ''}],
                               'pbcoreIdentifier': [{'source': 'example',
                                                     'text': '123'}],
                               'pbcoreTitle': [{'text': 'Title'}],
                               'xsi:schemaLocation': 'http://www.example.com/schema'}}


ValueError: Dict key must be str. Got 'int'

In [37]:
PBCore.model_validate(mvp)

PBCore(pbcoreDescriptionDocument=PBCoreDescriptionDocument(xsi_schemaLocation='http://www.example.com/schema', pbcoreAssetType=None, pbcoreAssetDate=None, pbcoreIdentifier=[PBCoreElement(ref=None, version=None, annotation=None)], pbcoreTitle=[PBCoreTitleItem(startTime=None, endTime=None, timeAnnotation=None, ref=None, version=None, annotation=None, titleType=None, titleTypeSource=None, titleTypeRef=None, titleTypeVersion=None, titleTypeAnnotation=None)], pbcoreSubject=None, pbcoreDescription=[PBCoreDescriptionItem(text='')], pbcoreCreator=None, pbcoreInstantiation=None, pbcoreAnnotation=None, pbcoreRelation=None, pbcoreCoverage=None, pbcoreAudienceLevel=None))